In [220]:
#from pyld import jsonld
import json

In [221]:
jsonld_input = ''
with open('./tds/lampThingSampleTD.json', 'r') as f:
#with open('./tds/poppyErgoJr_RobotArm_TD.json', 'r') as f:
    jsonld_input = json.load(f)

In [222]:
print(jsonld_input)
#print(json.dumps(jsonld.expand(jsonld_input), indent=2))

{'@context': ['https://www.w3.org/2019/wot/td/v1', {'saref': 'https://w3id.org/saref#'}], 'id': 'urn:dev:ops:32473-WoTLamp-1234', 'title': 'MyLampThing', '@type': 'saref:LightSwitch', 'actions': {'dim': {'@type': 'saref:levelcontrolfunction', 'input': {'type': 'object', 'properties': {'from': {'type': 'integer', 'minimum': 0, 'maximum': 100}, 'to': {'@type': 'example:dimTarget', 'type': 'integer', 'minimum': 0, 'maximum': 100}}, 'required': ['to']}, 'forms': [{'href': 'https://mylamp.example.com/dim'}]}}}


In [223]:
def extract_elements(jsonld_input, key):
    return jsonld_input.get(key, {})

def extract_actions(jsonld_input):
    return extract_elements(jsonld_input, 'actions')

def extract_events(jsonld_input):
    return extract_elements(jsonld_input, 'events')

def extract_properties(jsonld_input):
    return extract_elements(jsonld_input, 'properties')

In [224]:
actions = extract_actions(jsonld_input)
events = extract_events(jsonld_input)
properties = extract_properties(jsonld_input)

In [225]:
print(actions)
print(properties)
print(events)

{'dim': {'@type': 'saref:levelcontrolfunction', 'input': {'type': 'object', 'properties': {'from': {'type': 'integer', 'minimum': 0, 'maximum': 100}, 'to': {'@type': 'example:dimTarget', 'type': 'integer', 'minimum': 0, 'maximum': 100}}, 'required': ['to']}, 'forms': [{'href': 'https://mylamp.example.com/dim'}]}}
{}
{}


In [226]:
class Http_Form:
    def __init__(self, name, jsonld_description, isAction=False):            
        #print(jsonld_description)
        self.name = name
        self.endpoint = jsonld_description.get('href', '')
        self.content_type = jsonld_description.get('contentType', '')
        self.http_methods = []
        if 'htv:methodName' in jsonld_description:
            self.http_methods.append(jsonld_description.get('htv:methodName'))
        elif isAction:
            self.http_methods.append('POST') #default for action acc. to w3c
        elif 'op' in jsonld_description:
            # default for these operations according to w3c standard
            ops = jsonld_description.get('op')
            if type(ops) is str:
                ops = [ops]
            for op in ops:
                if op in {'readproperty', 'readallproperties', 'readmultipleproperties'}:
                    self.http_methods.append('GET')
                elif op in {'writeproperty', 'writeallproperties', 'writemultipleproperties'}:
                    self.http_methods.append('PUT')
                elif op == 'invokeaction':
                    self.http_methods.append('POST')
                elif op in {'subscribeevent', 'unsubscribeevent'}:
                    print(jsonld_description)
                    if 'subprotocol' in jsonld_description:
                        self.http_methods.append(jsonld_description.get('subprotocol'))
                    else:
                        print("ERROR: httpmethod of subscribeevent and unsubscribeevent not clearly defined")
                else:
                    print("ERROR: no http method assigned for op " + op)
                        
    def __str__(self):
        res = "Http_Form '" + self.name + "'\nendpoint: " + self.endpoint + "\ncontent_type: " + self.content_type + "\nhttp_methods: "
        for h in self.http_methods:
            res += h + ', '
        return  res
    

In [227]:
http_forms = []
for action_name, a in actions.items():
    for form in a.get('forms'):
        http_forms.append(('action', Http_Form(action_name, form, isAction=True)))

for property_name, p in properties.items():
    for form in p.get('forms'):
        http_forms.append(('property', Http_Form(property_name, form)))

for event_name, e in events.items():
    for form in e.get('forms'):
        http_forms.append(('event', Http_Form(event_name, form)))
for name, f in http_forms:
    print(name + " " + str(f))

action Http_Form 'dim'
endpoint: https://mylamp.example.com/dim
content_type: 
http_methods: POST, 


In [228]:
ui_outputs = {'general_data_output',
               'ordered_domain_output',
               'fixed_range_ordered_domain_output'
             }
ui_inputs = {'stateless_trigger_button',
               'stateless_forward_backward_buttons',
               'general_data_input',
               'ordered_domain_input',
               'ordered_domain_with_neutral_value_input',
               'operating_mode_input',
               'ordered_domain_fixed_range_input',
               'boolean_switch_input',
               'set_position_input',
               'move_input'
            }

In [229]:
# TODO: parse data types and together with Http_Form data, determine corresponding UI element
# property: type; if type==object, then it's nested!
# action: (input & output)->type; if type==object, then it's nested!
# event: data->type (this is the type that is being received)

In [230]:
def parse_properties(properties, http_forms):
    '''
        According to w3c standard: "Property instances are also instances of the class DataSchema. 
        Therefore, it can contain the type, unit, readOnly and writeOnly members, among others."
    '''
    for property_name, p in properties.items():
        for form in p.get('forms'):
            http_forms.append(('property', Http_Form(property_name, form)))
        property_type = p.get('type')
        if property_type in {'array', 'string', 'number', 'integer', 'boolean'}:
            # TODO: simple type, infer UI element for each form?
            print("property of simple type")
        elif property_type ==  'object':
            # TODO: recursive type checking
            print("property of type object, recursively parsing...")
        elif property_type == 'null':
            # TODO: define what happens here
            print("property of type null...")
            
def parse_events(events, http_forms):
    '''
        subscription (optional): Defines data that needs to be passed upon subscription.
        data (optional): Defines the data schema of the Event instance messages pushed by the Thing.
        cancellation (optional): Defines any data that needs to be passed to cancel a subscription
        
        e.g.
        
        "subscription": {
            "type": "object",
            "properties": {
                "callbackURL": {
                    "type": "string",
                    "format": "uri",
                    "description": "Callback URL provided by subscriber for Webhook notifications.",
                    "writeOnly": true
                },
                "subscriptionID": {
                    "type": "string",
                    "description": "Unique subscription ID for cancellation provided by WebhookThing.",
                    "readOnly": true
                }
            }
        },
        "data": {
            "type": "number",
            "description": "Latest temperature value that is sent to the callback URL."
        },
        "cancellation": {
            "type": "object",
            "properties": {
                "subscriptionID": {
                    "type": "integer",
                    "description": "Required subscription ID to cancel subscription.",
                    "writeOnly": true
                }
            }
        },
    '''
    for event_name, e in events.items():
        for form in e.get('forms'):
            http_forms.append(('event', Http_Form(event_name, form)))
        event_output_type = e.get('data').get('type')
        if property_type in {'array', 'string', 'number', 'integer', 'boolean'}:
            # TODO: simple type, infer UI element for each form?
            print("simple type")
            pass
        elif property_type ==  'object':
            # TODO: recursive type checking
            print("object type")
        elif prperty_type == 'null':
            # TODO: define what happens here
            print("null type")

In [231]:
def parse_uri_variables(jsonld):
    '''
        part of InteractionAffordance, same level as forms and connected to forms (no uriVariables without forms)
        
        e.g.
        
        "uriVariables": {
                "p" : { "type": "integer", "minimum": 0, "maximum": 16, "@type": "eg:SomeKindOfAngle" },
                "d" : { "type": "integer", "minimum": 0, "maximum": 1, "@type": "eg:Direction" }
        },
        "forms": [{
          "href" : "http://192.168.1.25/left{?p,d}",
          "htv:methodName": "GET"
        }]
    '''
    return None
    # not essential for now. TODO: implement later
    
def extract_data_description(jsonld):
    type_ = jsonld.get("type")
    min_ = jsonld.get("minimum")
    max_ = jsonld.get("maximum")
    ordered = False
    if type_ in {"integer", "number"}:
        # TODO: is boolean also ordered?
        ordered = True
    return {"type": type_, "ordered": ordered, "min": min_, "max": max_}

    
    
def map_array_type_input(jsonld):
    '''
    In general, arrays can be displayed and entered in a general data field by comma separated values.
    If the number of items is fixed and the types are known, a separate input field for each element could be displayed.
    The downside is, that the designators for each element is unknown, i.e. for RGB value in the example below it is clear
    if there is a single input field with description RGB value, 3 items, comma-separated. But it might be less clear
    if there are 3 input fields with only one title saying RGB values. Hence I would rather create a descriptive title
    and a single input field.
    
        "type": "array",
        "items" : {
            "type" : "number",
            "minimum": 0,
            "maximum": 255
        },
        "minItems": 3,
        "maxItems": 3
    '''
    data_description = extract_data_description(jsonld.get("items"))
    minItems = jsonld.get("minItems")
    maxItems = jsonld.get("maxItems")
    if minItems and maxItems and minItems == maxItems:
        # a fixed number of elements
        data_description["num_elements"] = minItems
    elif minItems and maxItems:
        data_description["num_elements"] = "min{}_max{}".format(minItems, maxItems)
    # TODO: complete implementation, array type is omitted for the PoC


# for both integer and number
def map_number_type_input(json_key, jsonld):
    '''
        "type": "number",
        "minimum": 0.0,
        "maximum": 100.0
    '''
    min_ = jsonld.get('minimum')
    max_ = jsonld.get('maximum')
    # TODO: handle case when it's not a range or only bounded to one side
    return generate_html_for_slider(json_key, min_, max_, min_)

def map_object_type_input(jsonld):
    # recursively parse types
    '''
            "type": "object",
            "properties": {
                "from": {
                    "type": "integer",
                    "minimum": 0,
                    "maximum": 100
                },
                "to": {
                    "type": "integer",
                    "minimum": 0,
                    "maximum": 100
                },
                "duration": {"type": "number"}
            },
            "required": ["to","duration"],
    '''
    props = jsonld.get('properties')
    ui_elements = []
    for p in props:
        ui_elements.append(map_to_input_ui_elements(p, props.get(p)))
    return ui_elements
        
    
def map_to_input_ui_elements(json_key, jsonld):
    t = jsonld.get('type')
    # TODO: also check for enum
    if t == 'object':
        return map_object_type_input(jsonld)
    elif t == 'integer' or  t == 'number':
        return map_number_type_input(json_key, jsonld)
    else:
        print("type not yet supported: {}".format(t))
    
def parse_actions(actions, http_forms):
    '''
            Heuristics for actions:
            - no input -> simple trigger
            - input -> - single input -> single UI_input_element (no trigger button, interaction with UI_input_element automatically triggers action, except for general (string) input)
                       - multiple inputs -> for each input UI_input_element and trigger button to trigger action
            - output -> sensor outputs; or could it also render back into a stateful actuator? 
                                        RE: might make sense for stateful actuators, i.e. request to change state
                                        to 10 and actuator does not change. Then input trigger-wheel will turn
                                        back to actual value. In the general case the output seems to be more something
                                        like a feedback. i.e. "success" or "error"
        "input": {
            "type": "object",
            "properties": {
                "from": {
                    "type": "integer",
                    "minimum": 0,
                    "maximum": 100
                },
                "to": {
                    "type": "integer",
                    "minimum": 0,
                    "maximum": 100
                },
                "duration": {"type": "number"}
            },
            "required": ["to","duration"],
        },
        "output": {"type": "string"},


        "actions": {
        "dim" : {
            "@type": "saref:levelcontrolfunction",
            "input": {
                "type": "object",
                "properties": {
                    "from": {
                        "type": "integer",
                        "minimum": 0,
                        "maximum": 100
                    },
                    "to": {
                        "@type": "example:dimTarget",
                        "type": "integer",
                        "minimum": 0,
                        "maximum": 100
                    }                },
                "required": ["to"]
            },
            "forms": [{
                "href": "https://mylamp.example.com/dim"
            }]
        }
    }
    '''
    overall_res = []
    for action_name, a in actions.items():
        print(action_name)
        print(a)
        input = a.get("input")
        output = a.get("output")
        # TODO: handle output
        res = map_to_input_ui_elements(action_name, input)
        if type(res) != list:
            res = [res]
        if len(res) > 1:
            # trigger button needed
            print("DEBUG: " + str(res))
            res.append(generate_html_for_button(action_name, action_name, a.get("forms")[0].get('href'), "{} arguments".format(len(res))))
        overall_res.append((action_name, res))
    return overall_res

action_parse_res = parse_actions(actions, http_forms)
html_base = Template(html_base_template.safe_substitute(header=container_style_string))

html_body = ''
html_script = ''
for action_name, html_ui_elements in action_parse_res:
    print("adding html elements for action {}".format(action_name))
    for html_part, js_part in html_ui_elements:
        html_body += html_part
        html_script += js_part
    # TODO: should different actions be put into separate <div> containers?

html_body = container_template.substitute(container_title=jsonld_input.get("title"), container_content=html_body)

html_body += script_template.substitute(js_content=html_script)

print(html_base.substitute(body=html_body))

dim
{'@type': 'saref:levelcontrolfunction', 'input': {'type': 'object', 'properties': {'from': {'type': 'integer', 'minimum': 0, 'maximum': 100}, 'to': {'@type': 'example:dimTarget', 'type': 'integer', 'minimum': 0, 'maximum': 100}}, 'required': ['to']}, 'forms': [{'href': 'https://mylamp.example.com/dim'}]}
DEBUG: [('<div><h5>from<h5><input id="from" type="range" min="0" max="100" value="0" class="slider" /><p>Value: <span id="from_output" /></p></div>', 'var from = document.getElementById("from");\n                            var from_output = document.getElementById("from_output");\n                            from_output.innerHTML = from.value;\n                            from.oninput = function() { from_output.innerHTML = this.value;}\n                            '), ('<div><h5>to<h5><input id="to" type="range" min="0" max="100" value="0" class="slider" /><p>Value: <span id="to_output" /></p></div>', 'var to = document.getElementById("to");\n                            var to_out

In [232]:
from string import Template
html_base_template = Template('''
                        <!DOCTYPE html>
                        <html>
                            <head>
                                $header
                            </head>
                            <body>
                                $body
                            </body>
                        </html>''')

container_template = Template('''<div class="container"><h4>$container_title</h4>$container_content</div>''')
container_style_string = '''<style>
                        .container {
                            width: fit-content;
                            border: 2px solid;
                        }
                    </style>'''

script_template = Template('''\n<script>$js_content</script>''')

def generate_html_for_slider(id, min, max, current):
    slider_template = Template('''<div><h5>$slider_title<h5><input id="$slider_id" type="range" min="$min" max="$max" value="$current" class="slider" /><p>Value: <span id="$output_id" /></p></div>''')
    slider_script_template = Template('''var $slider_id = document.getElementById("$slider_id");
                            var $output_id = document.getElementById("$output_id");
                            $output_id.innerHTML = $slider_id.value;
                            $slider_id.oninput = function() { $output_id.innerHTML = this.value;}
                            ''')
    html_body = slider_template.substitute(slider_title= id, slider_id=id, min=min, max=max, current=current, output_id=id+'_output')
    html_script = slider_script_template.substitute(slider_id=id, output_id=id+'_output')
    return html_body, html_script

def generate_html_for_general_input(designator, id):
    text_input_template = Template('''<label for="$id">$designator</label> <input type="text" id="$id" name="$id">''')
    return text_input_template.substitute(id=id, designator=designator), ""

def generate_html_for_button(id, text, trigger_api, trigger_payload):
    button_template = Template('''<button id=$id type="button" onclick="alert('sending request to $trigger_api with $trigger_payload')">$text</button>''')
    return button_template.substitute(id=id, trigger_api=trigger_api, trigger_payload=trigger_payload, text=text), ""


html_base = Template(html_base_template.safe_substitute(header=container_style_string))

html_body = ''
html_script = ''

# add slider 1
slider1_body, slider1_script = generate_html_for_slider('from', '0', '100', '0')
html_body += slider1_body
html_script += slider1_script
# add slider 2
slider2_body, slider2_script = generate_html_for_slider('to', '0', '100', '0')
html_body += slider2_body
html_script += slider2_script

html_body += generate_html_for_button('trigger_button', 'dim', "example.lamp.org", "{from:50, to:70}")[0]
html_body += generate_html_for_general_input("test:", "test'")[0]

html_body = container_template.substitute(container_title="Lamp Dimmer Thing", container_content=html_body)

html_body += script_template.substitute(js_content=html_script)

print(html_base.substitute(body=html_body))



                        <!DOCTYPE html>
                        <html>
                            <head>
                                <style>
                        .container {
                            width: fit-content;
                            border: 2px solid;
                        }
                    </style>
                            </head>
                            <body>
                                <div class="container"><h4>Lamp Dimmer Thing</h4><div><h5>from<h5><input id="from" type="range" min="0" max="100" value="0" class="slider" /><p>Value: <span id="from_output" /></p></div><div><h5>to<h5><input id="to" type="range" min="0" max="100" value="0" class="slider" /><p>Value: <span id="to_output" /></p></div><button id=trigger_button type="button" onclick="alert('sending request to example.lamp.org with {from:50, to:70}')">dim</button><label for="test'">test:</label> <input type="text" id="test'" name="test'"></div>
<script>var from = document.getEle